# Assistant-05<br/>Code Interpreter & Function calling

## Trading Assistant that can calculate portfolio values and email a report

## Notes

- 01/14/2024: Uploading a CSV file failed

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

## Email functionality

- Implemented using Logic Apps with an Http POST trigger

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import time
import requests
import oaihelper as framework
import yfinance as yf


## Get an OpenAI client
client = framework.get_openai_client()

### Get the latest stock price by ticker symbol using Yahoo Finance

In [2]:
def get_stock_price(symbol: str) -> float:
    try:
        stock = yf.Ticker(symbol)
        price = stock.history(period="1d")['Close'].iloc[-1]    
        return price
    except:
        return 0

### Use a Logic App HTTP POST trigger to email using Outlook

In [3]:
def send_post_request(json_payload):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(framework.email_URI, json=json_payload, headers=headers)
    if response.status_code == 202:
        print("Email sent to: " + json_payload['to'])  

### Prepare the tool for function calling

In [4]:
tools_list = [
{"type": "code_interpreter"},
{"type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
},
{
    "type": "function",
    "function": {

        "name": "send_email",
        "description": "sends an email to a recipient(s).",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "The email(s) the email should be sent to."
                },
                "content": {
                    "type": "string",
                    "description": "The content of the email."
                }
            },
            "required": ["to", "content"]
        }
    }
}]

In [5]:
csv_file = framework.upload_file(client,"../data/fta-portfolio.csv")

### Create the assistant

In [6]:
framework.clear_shelves()

## Create an assistant
stock_assistant = framework.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list,                                                  
                                                  model=common.gpt_deployment_name,
                                                  file_ids=[csv_file.id])

Added assistant:  asst_jUGXQfgJqX3vIU0Czd4Yr27R 1


### Create a thread, put a message on the thread, and view the initial status

In [7]:
# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Based on the provide portfolio and using latest stock prices, please email me an HTML formatted report at alemor@microsoft.com with the details for each stock, and the best and worst performing stocks in my portfolio."
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=stock_assistant.id,
    instructions="Today's date and time is: " + framework.get_localized_datetime() + ".\n\n"
)

#print(run.model_dump_json(indent=4))

### Process the message, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [8]:
while True:
    # Wait for 1 seconds
    time.sleep(5)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    #print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        ## Cleanup
        print(client.beta.threads.delete(thread.id))
        print(client.beta.assistants.delete(stock_assistant.id))
        print(client.files.delete(csv_file.id))

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "get_stock_price":
                output = get_stock_price(symbol=arguments['symbol'])
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": output
                })
            elif func_name == "send_email":
                print("Sending email...")
                email_to = arguments['to']
                email_content = arguments['content']
                send_post_request({'to': email_to,'content':email_content})
                
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": "Email sent"
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)

Waiting for the Assistant to process...
Waiting for the Assistant to process...
Waiting for the Assistant to process...
Waiting for the Assistant to process...
Waiting for the Assistant to process...
Waiting for the Assistant to process...
Assistant: Once again, I apologize for the inconvenience. Unfortunately, I am unable to access the file using this approach as well. Since these methods are not yielding the desired result, I would suggest downloading the file and opening it on your local machine to see if you can identify its format and contents. Once you have verified the file's format, I can continue with a more targeted method to parse and utilize the data.

Please check the file locally and let me know the format and the structure of the data (such as the column names and the type of information contained within), or if possible, provide a sample of the data. With this information, I can proceed with creating the required stock performance report and sending the email.
Assistant